In [3]:
import pandas as pd
import tensorflow as tf
import keras as ks
import numpy as np
from keras.layers import Dense, Dropout, Conv2D, Flatten, Activation
from keras.models import Sequential
import matplotlib.pylab as plt

Using TensorFlow backend.


In [4]:
# Create new variables to be used in Keras and the CNN

# number of items to use for training
BATCH_SIZE = 10 

# Number of identifying classes 
#   WE have two, Bloom and no bloom 1/0
NUM_CLASSES = 2 

# number of times to repeat process
EPOCHS = 20

df = pd.read_csv('../../data/cleaned/site1_malin.csv')
df.head()

,date (mm.dd.yyyy),Depth [m],T [deg C],DO [%],SPC [micro S/cm],NH4 [mg/l],NO3 [mg/l],PO4 [mg/l],Chl tot [micro g/l],Chl cyano [micro g/l],Chl green Al. [micro g/l],turb [FTU],Secci Depth [m],pH
0,6/1/2016,0.0,20.8,127.3,748,0.877326755,0.162558848,0.012251796,39.8,3.2,36.6,5.1,0.8,8.63
1,6/1/2016,0.5,20.8,127.0,748,-,-,-,55.4,3.7,51.7,4.8,-,8.65
2,6/1/2016,1.0,20.4,131.4,756,-,-,-,45.6,3.0,42.6,4.9,-,8.66
3,6/1/2016,2.0,18.0,85.4,775,-,-,-,41.0,3.9,37.1,5.0,-,8.25
4,6/1/2016,3.0,15.8,4.0,786,1.050011213,0.270562171,0.031263202,54.6,5.8,48.8,5.8,-,7.89


In [24]:
target = df['Chl cyano [micro g/l]'].apply(lambda x : 1 if x > 20 else 0)
dataset = df.drop(columns=['NH4 [mg/l]', 'NO3 [mg/l]', 'PO4 [mg/l]', 'Secci Depth [m]','Chl cyano [micro g/l]'])
dataset['bloom'] = target
dataset.head(5)

,date (mm.dd.yyyy),Depth [m],T [deg C],DO [%],SPC [micro S/cm],Chl tot [micro g/l],Chl green Al. [micro g/l],turb [FTU],pH,bloom
0,6/1/2016,0.0,20.8,127.3,748,39.8,36.6,5.1,8.63,0
1,6/1/2016,0.5,20.8,127.0,748,55.4,51.7,4.8,8.65,0
2,6/1/2016,1.0,20.4,131.4,756,45.6,42.6,4.9,8.66,0
3,6/1/2016,2.0,18.0,85.4,775,41.0,37.1,5.0,8.25,0
4,6/1/2016,3.0,15.8,4.0,786,54.6,48.8,5.8,7.89,0


In [25]:
from datetime import datetime

dataset['date (mm.dd.yyyy)'] = pd.to_datetime(dataset['date (mm.dd.yyyy)'])

#converts the date object to a numerical representation of that object
dataset['date (mm.dd.yyyy)'] = (dataset['date (mm.dd.yyyy)'] - dataset['date (mm.dd.yyyy)'].min()) / np.timedelta64(1,'D')
dataset.dtypes

date (mm.dd.yyyy)            float64
Depth [m]                    float64
T [deg C]                    float64
DO [%]                       float64
SPC [micro S/cm]               int64
Chl tot [micro g/l]          float64
Chl green Al. [micro g/l]    float64
turb [FTU]                   float64
pH                           float64
bloom                          int64
dtype: object

In [26]:
# lets try to normalize this now....
from sklearn.preprocessing import MinMaxScaler
dataset_columns = ['date (mm.dd.yyyy)','Depth [m]','T [deg C]', 'DO [%]','SPC [micro S/cm] ',
'Chl tot [micro g/l]','Chl green Al. [micro g/l]', 'turb [FTU]','pH ','bloom']
scaler = MinMaxScaler()
ds_scaled = scaler.fit_transform(dataset)
dataset = pd.DataFrame(ds_scaled,columns=dataset_columns)
dataset.describe()


/home/mytstikyl/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,date (mm.dd.yyyy),Depth [m],T [deg C],DO [%],SPC [micro S/cm],Chl tot [micro g/l],Chl green Al. [micro g/l],turb [FTU],pH,bloom
count,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000
mean,0.500000,0.433333,0.630545,0.577551,0.402597,0.332460,0.260335,0.137371,0.483394,0.300000
std,0.303492,0.360654,0.249957,0.205032,0.206691,0.190768,0.212422,0.176529,0.211168,0.460355
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.238095,0.166667,0.556667,0.526077,0.219780,0.219836,0.107899,0.034502,0.316667,0.000000
50%,0.500000,0.333333,0.713333,0.590703,0.357143,0.279141,0.192182,0.059955,0.460000,0.000000
75%,0.761905,0.666667,0.771667,0.670068,0.571429,0.374233,0.333062,0.160633,0.680000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [169]:
def windows(dataset, window_size):
    start = 0
    while start < dataset.shape[0]: 
        yield (int(start), int(start+window_size))
        # shift the window one block of time
        start += 1 
        if start % 5 == 0:
            print('Window Segmentation {0:.2f}% done'.format((start / dataset.shape[0]) * 100))


'''

Segments the dataset based on the parameters that are passed in.
@param dataset - the dataset to segment into window
@param columns - the array of columns from the dataset to be looked at
@param window_size - the size of the window you would like to be looked at. Defualt is 10

'''
def segment_dataset(dataset, columns, target, window_size=10):    
    print('WINDOW SIZE',window_size)
    print('NUMBER OF COULUMNS',len(columns))
    segments = np.empty((0, window_size, len(columns)))
    labels = np.empty((0))
    count = 0
    for (start, end) in windows(dataset, window_size):
        count+=1
        values = dataset[columns][start:end]
        if(values.shape[0] == window_size):
            segments = np.vstack([segments, np.stack([values])])
            #takes the smaller of the two most common target variables in the array
            labels = np.append(labels, dataset[target][start:end].mode().iloc[0])
        else:
            print("No more Windows available... Exiting")
            break
    return (segments, labels)

In [170]:
feature_columns = dataset_columns[:-1]
(segments, labels) = segment_dataset(dataset, feature_columns, 'bloom', 10)
print('done')

WINDOW SIZE 10
NUMBER OF COULUMNS 9
Window Segmentation 4.55% done
Window Segmentation 9.09% done
Window Segmentation 13.64% done
Window Segmentation 18.18% done
Window Segmentation 22.73% done
Window Segmentation 27.27% done
Window Segmentation 31.82% done
Window Segmentation 36.36% done
Window Segmentation 40.91% done
Window Segmentation 45.45% done
Window Segmentation 50.00% done
Window Segmentation 54.55% done
Window Segmentation 59.09% done
Window Segmentation 63.64% done
Window Segmentation 68.18% done
Window Segmentation 72.73% done
Window Segmentation 77.27% done
Window Segmentation 81.82% done
Window Segmentation 86.36% done
Window Segmentation 90.91% done
No more Windows available... Exiting
done


In [171]:
segments.shape

(101, 10, 9)

In [172]:
labels.shape

(101,)

In [173]:
labels

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])